In [1]:
import rospy
import tf
from tf.transformations import quaternion_matrix
import actionlib
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningActionFeedback, Custom
from nav_msgs.msg import Odometry
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

%matplotlib widget

# Global variables for robot position and status
latestX = 0.0
latestY = 0.0
state = ""
set_target = []
canceled_target = []
goals = 0
first = True
reached_targets = 0
not_reached_targets = 0
current_target = None

# Initialize the output widgets
output_info = widgets.Output()
output_targets = widgets.Output()
output_bar = widgets.Output()


In [2]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'g^', markersize=3, linestyle='-', linewidth=1)
        self.target_marker, = plt.plot([], [], 'ro', markersize=8, linestyle='')  # Target marker
        self.x_data, self.y_data = [], []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.target_marker

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        global latestX, latestY
        latestX = msg.pose.pose.position.x
        latestY = msg.pose.pose.position.y

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        if current_target:
            self.target_marker.set_data(current_target[0], current_target[1])
        return self.ln, self.target_marker

vis = Visualiser()
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, save_count=100)
plt.show(block=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
def feedback_callback(feed):
    global state, reached_targets, first
    state = feed.feedback.stat
    if state == "Target reached!" and first:
        reached_targets += 1
        first = False

def plot_bar(ax, x, y):
    x_numeric = range(len(x))  # Convert x values to numerical
    ax.bar(x_numeric, y, width=0.4, align='center')
    ax.set_xticks(x_numeric)
    ax.set_xticklabels(x)
    ax.set_xlabel('Targets')
    ax.set_ylabel('Count')
    ax.set_title('Targets Status')
    ax.set_ylim(0, 10)  # Set the y-axis limit to 10

def update_bar():
    y_values = [reached_targets, not_reached_targets]
    with output_bar:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        plot_bar(ax, ['Reached Targets', 'Not Reached Targets'], y_values)
        plt.show()


In [4]:
def update_info():
    with output_info:
        clear_output(wait=True)
        print(f"The position of X is: {latestX}")
        print(f"The position of Y is: {latestY}")
        print(f"The status: {state}")

def update_targets():
    with output_targets:
        clear_output(wait=True)
        print("\nSet targets:")
        for pair in set_target:
            print(pair)

        print("\nCanceled targets:")
        for pair in canceled_target:
            print(pair)

        print(f"\nReached Targets: {reached_targets}")
        print(f"Not Reached Targets: {not_reached_targets}")


In [5]:
def main():
    global pub, goals, x_input, y_input, current_target

    rospy.init_node('actcl')

    # Action client for "reaching_goal"
    ac = actionlib.SimpleActionClient('reaching_goal', PlanningAction)

    rospy.loginfo("Waiting for action server to start.")
    ac.wait_for_server()
    rospy.loginfo("Action server started, sending goal.")

    # Subscribers and publisher
    rospy.Subscriber("reaching_goal/feedback", PlanningActionFeedback, feedback_callback)
    rospy.Subscriber("odom", Odometry, vis.odom_callback)
    pub = rospy.Publisher("custom_pos_vel", Custom, queue_size=1)

    # Create widgets
    x_input = widgets.FloatText(description='X:', layout=widgets.Layout(width='200px'))
    y_input = widgets.FloatText(description='Y:', layout=widgets.Layout(width='200px'))

    def choice1(b):
        global x_input, y_input, goals, first, current_target
        x = float(x_input.value)
        y = float(y_input.value)

        goal = PlanningGoal()
        goal.target_pose.pose.position.x = x
        goal.target_pose.pose.position.y = y

        ac.send_goal(goal)
        new_pair = (x, y)
        set_target.append(new_pair)
        goals += 1
        current_target = new_pair  # Update current target
        first = True
        update_info()
        update_targets()
        update_bar()

    def choice2(b):
        global goals, not_reached_targets, first, current_target
        if set_target and state != "Target reached!" and goals > 0:  # Ensure there's a target to cancel
            ac.cancel_goal()
            canceled_target.append(set_target[-1])
            goals -= 1
            not_reached_targets += 1
            first = True
            current_target = None  # Reset current target if canceled
            update_info()
            update_targets()
            update_bar()

    def choice3(b):
        update_info()
        update_targets()
        update_bar()

    _button = widgets.Button(description='Set Target', layout=widgets.Layout(width='150px'))
    _cancel_target_button = widgets.Button(description='Cancel Target', layout=widgets.Layout(width='150px'))
    _info_target_button = widgets.Button(description='Info Target', layout=widgets.Layout(width='150px'))

    # Layout
    _inputs = widgets.HBox([x_input, y_input])
    _buttons = widgets.HBox([_button, _cancel_target_button, _info_target_button])
    display(widgets.VBox([_inputs, _buttons, output_info, output_targets, output_bar]))

    # Register button callbacks
    _button.on_click(choice1)
    _cancel_target_button.on_click(choice2)
    _info_target_button.on_click(choice3)

if __name__ == '__main__':
    main()


[INFO] [1717179985.603549, 0.000000]: Waiting for action server to start.
[INFO] [1717179985.687415, 2445.279000]: Action server started, sending goal.
